In [1]:
import time
import multiprocessing
import qpo_search as q
import numpy as np
import scipy.fft as fft
import matplotlib.pyplot as plt
import os 
import glob
from scipy import stats
from scipy import optimize
from astropy.io import fits
from scipy.optimize import curve_fit
from astropy.io import ascii
from astropy.table import Table
import csv
import math
import stingray
import lightcurve
from stingray import Lightcurve
#from stingray import Powerspqpoectrum
from stingray import AveragedPowerspectrum
from stingray import Crossspectrum
from stingray.exceptions import StingrayError
from stingray import AveragedCrossspectrum
from more_itertools import locate
import matplotlib.pylab as pl
import matplotlib.gridspec as gridspec
import scipy
from scipy import stats
from scipy import optimize
from scipy.stats import f
import scipy.integrate as integrate
import scipy.special as special
from scipy.special import erfinv, erf
#from sklearn import preprocessing
from pynverse import inversefunc
import pynverse
import warnings
warnings.filterwarnings('ignore')
from concurrent.futures import ThreadPoolExecutor
from joblib import Parallel, delayed
from multiprocessing import Process
import threading
import multiprocessing as mp
from functools import partial
import pandas as pd
from scipy import optimize
from astropy.io import fits
from scipy.optimize import curve_fit
from astropy.io import ascii
from astropy.table import Table
import csv
import math
import stingray
import lightcurve
from stingray import Lightcurve
from stingray import Powerspectrum
from stingray import AveragedPowerspectrum
from stingray import Crossspectrum
from stingray.exceptions import StingrayError
from stingray import AveragedCrossspectrum
from more_itertools import locate
import matplotlib.pylab as pl
import matplotlib.gridspec as gridspec
import scipy.integrate as spi
import pynverse
from pynverse import inversefunc
import functools
from functools import partial
import warnings
from multiprocessing import Pool
warnings.filterwarnings('ignore')

# Using concurrent.futures to parallelise instead of pool.map because we need a true shared memory for objects more complex than integers.


In [4]:
#Example of concurrent futures

import concurrent.futures
import time
import random

def mail_letter(letter):
    duration = random.randint(1, 5)
    print(f"Started mailing letter {letter} (duration: {duration}s)")
    time.sleep(duration)
    print(f"Finished mailing letter {letter}")
    return f"Letter {letter} mailed"

if __name__ == '__main__':
    letters = ['A', 'B', 'C', 'D', 'E']

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {executor.submit(mail_letter, letter): letter for letter in letters}

        for future in concurrent.futures.as_completed(futures):
            letter = futures[future]
            result = future.result()
            print(f"Result: {result}")

Started mailing letter A (duration: 3s)
Started mailing letter B (duration: 5s)
Started mailing letter C (duration: 5s)
Started mailing letter D (duration: 1s)
Started mailing letter E (duration: 5s)
Finished mailing letter D
Result: Letter D mailed
Finished mailing letter A
Result: Letter A mailed
Finished mailing letter B
Finished mailing letter C
Finished mailing letter E
Result: Letter B mailed
Result: Letter C mailed
Result: Letter E mailed


In [18]:
def run_new_null_concurrent(source_name,file12,file3,mue_file,GTI_file,run_num_tot,cpu_num,Pmin,Pmax,bin_length,seg_length,fmin,fmax,modmin,modmax,mod_bin_number,p,psi):
    with fits.open(str(file12)) as hdu:
            data_12=hdu[1].data  #reading in DU1+DU2
            mod_angles=data_12.field('PHI')
            #print(mod_angles)
    with fits.open(str(file3)) as hdu2:
            data_header=hdu2[1].header #reading in header
            data_3=hdu2[1].data #reading in DU3
            #print(data_3)
            #energy cut
            index_energy=list(locate(data_12.field('PI'), lambda x: Pmin < x < Pmax))  
            data_12=data_12[index_energy]
            index_energy_3=list(locate(data_3.field('PI'), lambda x: Pmin < x < Pmax))
            data_3=data_3[index_energy_3]

            #quality factor cut
            index_qual_12=list(locate(data_12.field('QUAL'), lambda x: x==1)) 
            data_12=data_12[index_qual_12]
            index_qual_3=list(locate(data_3.field('QUAL'), lambda x: x==1))  
            data_3=data_3[index_qual_3]

            #TSTART=data_header['TSTART']
            #TSTART=list([TSTART,0])
            #TSTOP=data_header['TSTOP']
            #curve_duration=TSTOP-TSTART

            TIME=data_12.field('TIME')
            TIME_3=data_3.field('TIME')
            #print(TIME_3)

            GTI=list(np.loadtxt(str(GTI_file)))
            #Lightcurves for norm
            lightcurve_12=Lightcurve.make_lightcurve(TIME,dt=bin_length,gti=GTI)
            lightcurve_12.apply_gtis()
            #print('lc 12')
            #print(len(lightcurve_12.counts))
            #plt.figure()
            #plt.plot(lightcurve_12.time,lightcurve_12.counts)
            #plt.show()


            lc_ref=Lightcurve.make_lightcurve(TIME_3,dt=bin_length,gti=GTI)
            lc_ref.apply_gtis()
            #print('lc ref')
            #print(len(lc_ref.counts))
            #plt.figure()
            #plt.plot(lc_ref.time,lc_ref.counts)
            #plt.show()

            #Cross spec for norm ie over all mod angle bins

            avg_cs = AveragedCrossspectrum.from_lightcurve(lightcurve_12,lc_ref,seg_length,norm='frac')
            #print('avg cs')


            norm_power_real=avg_cs.power.real  #cross spec properties
            norm_power_im=avg_cs.power.imag
            norm_freq=avg_cs.freq


            #real power of all mod cross spec (for norm constant and 2019 errorbars)
            norm_d = {'real_power': np.array(avg_cs.power.real), 'all_fourier_freq': np.array(avg_cs.freq)} #total pwr and freq in dataset
            df_norm = pd.DataFrame(data=norm_d)
            selected_rows_norm = df_norm[(df_norm['all_fourier_freq'] >= fmin) & (df_norm['all_fourier_freq'] <= fmax)] #selecting freq range
            av_power_norm_real=selected_rows_norm['real_power'].mean() #calculating mean pwr

            #av_power_norm_array.append(av_power_norm_real)

            #calculating normalisation constant

            norm_factor=(np.sqrt((fmax-fmin))/np.sqrt(av_power_norm_real))
            norm_factor_array=[]
            norm_factor_array.append(norm_factor) 
            np.savetxt('Results/newnull_norm_cs_'+str(source_name)+'_freqs_'+str(fmin)+'_'+str(fmax)+'_'+str(bin_length)+'_'+str(seg_length)+'.txt',norm_factor_array)


            #making a list of mod angle bins to select over
           # mod_minimum=np.radians(modmin)
           # mod_maximum=np.radians(modmax)
           # aspace=np.linspace(mod_minimum,mod_maximum,mod_bin_number+1)
            #mod_angle_list=[(aspace[i-1],aspace[i]) for i in range(len(aspace))]  #making a list of mod angle bins to select over
            #mod_angle_list.pop(0) #removing the dodger first one

            mue=np.loadtxt(mue_file)[:,2]
            modulation_factor_12=np.array([mue[int(i)] for i in data_12.field('PI')])
            #print(modulation_factor_12)
            modulation_factor_3=np.array([mue[int(i)] for i in data_3.field('PI')])



            #what i need to transfer over to the parallelised 

            #mod_angle_list
            #modulation_factor_12
            #modulation_factor_3
            #GTI
            #TIME
            #TIME_3
            #TSTART


         #converting the manager stuff to array stuff to see if theres an increase in speed etc   

          #  manager = multiprocessing.Manager()
    
            #print(mod_angle_list)
            #mod_angle_list = Array('d',mod_angle_list)
          #  modulation_factor_12 =manager.Array('d',modulation_factor_12)
          #  modulation_factor_3 = manager.Array('d',modulation_factor_3)
            #GTI=Array('d',GTI)
            #TIME=[1,2,3]
            #TIME_3=[4,5,6]
          #  TIME=manager.Array('d',TIME)
          #  TIME_3=manager.Array('d',TIME_3)
            #TSTART=manager.Array('d',TSTART)
            iterables=np.array(range(cpu_num))
            new_null_concurrent_partial=partial(new_null_concurrent,mod_angles=mod_angles,modulation_factor_12=modulation_factor_12,modulation_factor_3=modulation_factor_3,GTI_file=GTI_file,TIME=TIME,TIME_3=TIME_3,source_name=source_name,run_num_tot=run_num_tot,cpu_num=cpu_num,Pmin=Pmin,Pmax=Pmax,bin_length=bin_length,seg_length=seg_length,fmin=fmin,fmax=fmax,modmin=modmin,modmax=modmax,mod_bin_number=mod_bin_number,p=p,psi=psi)
            
            
            with concurrent.futures.ThreadPoolExecutor() as executor:
                futures = {executor.submit(new_null_concurrent, iterable): iterable for iterable in iterables}
#
#                for future in concurrent.futures.as_completed(futures):
#                    letter = futures[future]
#                    result = future.result()
#                    print(f"Result: {result}")


        
        
        
        
        
        #  iterable=np.array(range(cpu_num))
            #print(iterable)
    
#iterable,modulation_factor_12,modulation_factor_3,GTI_file,TIME,TIME_3,source_name,run_num_tot,cpu_num,Pmin,Pmax,bin_length,seg_length,fmin,fmax,modmin,modmax,mod_bin_number,p,psi)    
    
    #pool = multiprocessing.Pool()
    #pool.map(func, [0, 1]
    
   # new_null_partial=partial(new_null,source_name=source_name,file12=file12,file3=file3,mue_file=mue_file,GTI_file=GTI_file,run_num_tot=run_num_tot,cpu_num=cpu_num,Pmin=Pmin,Pmax=Pmax,bin_length=bin_length,seg_length=seg_length,fmin=fmin,fmax=fmax,modmin=modmin,modmax=modmax,mod_bin_number=mod_bin_number,p=p,psi=psi)
    
   

          #  with Pool(processes=cpu_num) as pool:
              #result = pool.map(new_null_partial,)
           #   result=pool.map(new_null_manager_partial,iterable)


In [15]:
def new_null_concurrent(iterable,mod_angles,modulation_factor_12,modulation_factor_3,GTI_file,TIME,TIME_3,source_name,run_num_tot,cpu_num,Pmin,Pmax,bin_length,seg_length,fmin,fmax,modmin,modmax,mod_bin_number,p,psi):
    #TSTART=TSTART[0]
    print(iterable)
    #print(len(modulation_factor_12))
    #print(len(modulation_factor_3))
    GTI=list(np.loadtxt(str(GTI_file)))
    #print(GTI)
    TIME=np.array([TIME])
    TIME_3=np.array([TIME_3])
    #mod_angle_array_12=np.array([mod_angles])
    #print(mod_angles)
    #print(len(TIME_3))
    #print(type(TIME_3))
    #print(TIME_3)
    #GTI=np.array([GTI])
    #print(type(GTI))
    #print(bin_length)
#making ref lightcurve


    lc_ref=Lightcurve.make_lightcurve(TIME_3,dt=bin_length,gti=GTI)
    print('lc ref done')
    lc_ref.apply_gtis()
    run_num=0


    mod_G_stacked=[]
    arg_G_stacked=[]
    d_mod_G_stacked=[]
    d_arg_G_stacked=[]

    G_running_sum=np.zeros(mod_bin_number,dtype=complex)
    dG_running_sum=np.zeros(mod_bin_number)
    norm_power_running_sum=np.zeros(mod_bin_number)

    mod_minimum=np.radians(modmin)
    mod_maximum=np.radians(modmax)
    aspace=np.linspace(mod_minimum,mod_maximum,mod_bin_number+1)
    mod_angle_list=[(aspace[i-1],aspace[i]) for i in range(len(aspace))]  #making a list of mod angle bins to select over
    mod_angle_list.pop(0) 


    while run_num<run_num_tot//cpu_num:
        print('in while loop')
        run_num+=1
        #print()
        mod_angle_array_12=[]
        mod_angle_array_3=[]
        #print('{} core {} run'.format(iterable,run_num))
        #for mod_fac in modulation_factor_12:
#                   #print(mod_fac)
#
         #   cdf_12=(lambda phi: phi - modmin + 0.5*mod_fac*p * ( np.sin( 2*phi - 2*psi ) - np.sin( 2*modmin - 2*psi ) ))
         #   cdf_min_12=cdf_12(np.radians(modmin))
         #   cdf_max_12=cdf_12(np.radians(modmax))
         #   random_12=np.random.uniform(cdf_min_12,cdf_max_12)
         #   mod_angle_array_12.append(inversefunc(cdf_12, y_values=random_12))
            #print(mod_angle_array_12)

#              print('mod angles calculated')




       # for mod_fac in modulation_factor_3:
       #     cdf_3=(lambda i: (1/np.pi) * (i +  (mu*p*0.5) * (np.sin((2*i)-(2*psi)) + np.sin(2*psi))))
       #     cdf_min_3=cdf(np.radians(0))
       #     cdf_max_3=cdf(np.radians(180))
       #     random_3=np.random.uniform(cdf_min,cdf_max)
       #     mod_angle_array_3.append(inversefunc(cdf_3, y_values=random_3))


        av_power_G_array=[]
        m_array=[]
        av_power_array_mod_ps_12=[]
        av_power_array_mod_ps_3=[]
        av_power_array_real=[]
        av_power_array_im=[]
        #mod_angle_array_12=np.array(])

        print(len(mod_angles))
        print(len(TIME))
        for i in mod_angle_list:
            mod_min=i[0]
            print(np.degrees(mod_min))
            mod_max=i[1]

            print(np.degrees(mod_max))

            index_mod_angle=[mod_min<=k<=mod_max for k in mod_angles] #define the index over mod angle
            TIME_12=TIME[index_mod_angle] #selecting/indexing the photons that meet the criteria of this mod angle range

           # print('len data bin 12',len(data_bin_12))
            #TIME_12=data_bin_12['TIME']
            #print(TIME_12)

            #making subject lightcurve
            lc_12=Lightcurve.make_lightcurve(TIME_12,dt=bin_length,gti=GTI)
            lc_12.apply_gtis()

            #print('lc sub')

            #plt.figure()
            #plt.plot(lc_12.time,lc_12.counts)
            #plt.show()
            #making powerspectrum for errorbars
                #power spectrum used for frac rms errorbars
            ps_12=Powerspectrum.from_lightcurve(lc_12,seg_length,norm='frac')
            ps_3=Powerspectrum.from_lightcurve(lc_ref,seg_length,norm='frac')

            #making averagd cross spectrum
            cs = AveragedCrossspectrum.from_lightcurve(lc_12,lc_ref,seg_length,norm='frac') #making averagd cross spec
            mod_G=np.sqrt((cs.power.real)**2+(cs.power.imag**2))
            m=cs.m



            fourier_f=cs.freq
            real_power=cs.power.real   #real and im power of G
            im_power=cs.power.imag

           #averaging the modulus of G to use in drms

            #d_G_ = {'G': np.array(mod_G), 'fourier_freq': np.array(cs.freq)}
            #df_G = pd.DataFrame(data=d_G_)
            #selected_rows_G = df_G[(df_G['fourier_freq'] >= fmin) & (df_G['fourier_freq'] <= fmax)]

            #print(selected_rows_G[:1000])

            freq_index=list(locate(np.array(cs.freq), lambda x: fmin <= x <= fmax))

            mod_G_selected=mod_G[freq_index]
            #print(mod_G_selected[:1000])


            av_power_G=mod_G_selected.mean()
            #print('modulus of G mean: {}'.format(av_power_G))
            av_power_G_array.append(av_power_G)
            #np.savetxt('Results/av_G_mod'+str(source_name)+'_'+str(mod_bin_number)+'_bins_freqs_'+str(fmin)+'_'+str(fmax)+'_'+str(bin_length)+'_'+str(seg_length)+'.txt',av_power_G_array)




           #averaging the power spectrum over fourier freq for drms errorbars

            mod_ps_12=ps_12.power.real #this is because the poewr spectrum is real by definition!
            mod_ps_3=ps_3.power.real

                        #d_mod_ps_12 = {'mod_power_ps_12': np.array(mod_ps_12), 'fourier_freq': np.array(ps_12.freq)}
            #df_mod_ps_12 = pd.DataFrame(data=d_mod_ps_12)
            #selected_rows_mod_ps_12 = df_mod_ps_12[(df_mod_ps_12['fourier_freq'] >= fmin) & (df_mod_ps_12['fourier_freq'] <= fmax)]
            selected_rows_mod_ps_12=mod_ps_12[freq_index]
            av_power_mod_ps_12=selected_rows_mod_ps_12.mean()
            av_power_array_mod_ps_12.append(av_power_mod_ps_12)


            #d_mod_ps_3 = {'mod_power_ps_3': np.array(mod_ps_3), 'fourier_freq': np.array(ps_3.freq)}
            #df_mod_ps_3 = pd.DataFrame(data=d_mod_ps_3)
            #selected_rows_mod_ps_3 = df_mod_ps_3[(df_mod_ps_3['fourier_freq'] >= fmin) & (df_mod_ps_3['fourier_freq'] <= fmax)]
            selected_rows_mod_ps_3=mod_ps_3[freq_index]
            av_power_mod_ps_3=selected_rows_mod_ps_3.mean()
            av_power_array_mod_ps_3.append(av_power_mod_ps_3)



            #Average real power over frequency range
            #d_real = {'real_power': np.array(real_power), 'fourier_freq': np.array(cs.freq),'dPco':np.array(dPco)}
            #df_real = pd.DataFrame(data=d_real)
            #selected_rows_real = df_real[(df_real['fourier_freq'] >= fmin) & (df_real['fourier_freq'] <= fmax)]
            selected_rows_real=real_power[freq_index]
            av_power_real=selected_rows_real.mean()
            av_power_array_real.append(av_power_real)
            #np.savetxt('Results/G_av_real_'+str(source_name)+'_'+str(mod_bin_number)+'_bins_freqs_'+str(fmin)+'_'+str(fmax)+'_'+str(bin_length)+'_'+str(seg_length)+'.txt',av_power_array_real)


            #Average imaginary power over frequency
            #d_im = {'im_power': np.array(im_power), 'fourier_freq': np.array(cs.freq),'dPcoim': np.array(dPcoim)}
            #df_im = pd.DataFrame(data=d_im)
            #selected_rows_im = df_im[(df_im['fourier_freq'] >= fmin) & (df_im['fourier_freq'] <= fmax)]
            selected_rows_im=im_power[freq_index]
            av_power_im=selected_rows_im.mean()
            av_power_array_im.append(av_power_im)
            #np.savetxt('Results/G_av_im_'+str(source_name)+'_'+str(mod_bin_number)+'_bins_freqs_'+str(fmin)+'_'+str(fmax)+'_'+str(bin_length)+'_'+str(seg_length)+'.txt',av_power_array_im)


            #everything i want from a single run    
            #mod_G_av=np.sqrt(np.array(av_power_array_real)**2+np.array(av_power_array_im)**2)
def new_null_concurrent(iterable,mod_angles,modulation_factor_12,modulation_factor_3,GTI_file,TIME,TIME_3,source_name,run_num_tot,cpu_num,Pmin,Pmax,bin_length,seg_length,fmin,fmax,modmin,modmax,mod_bin_number,p,psi):
    #TSTART=TSTART[0]
    print(iterable)
    #print(len(modulation_factor_12))
    #print(len(modulation_factor_3))
    GTI=list(np.loadtxt(str(GTI_file)))
    #print(GTI)
    TIME=np.array([TIME])
    TIME_3=np.array([TIME_3])
    #mod_angle_array_12=np.array([mod_angles])
    #print(mod_angles)
    #print(len(TIME_3))
    #print(type(TIME_3))
    #print(TIME_3)
    #GTI=np.array([GTI])
    #print(type(GTI))
    #print(bin_length)
#making ref lightcurve


    lc_ref=Lightcurve.make_lightcurve(TIME_3,dt=bin_length,gti=GTI)
    print('lc ref done')
    lc_ref.apply_gtis()
    run_num=0


    mod_G_stacked=[]
    arg_G_stacked=[]
    d_mod_G_stacked=[]
    d_arg_G_stacked=[]

    G_running_sum=np.zeros(mod_bin_number,dtype=complex)
    dG_running_sum=np.zeros(mod_bin_number)
    norm_power_running_sum=np.zeros(mod_bin_number)

    mod_minimum=np.radians(modmin)
    mod_maximum=np.radians(modmax)
    aspace=np.linspace(mod_minimum,mod_maximum,mod_bin_number+1)
    mod_angle_list=[(aspace[i-1],aspace[i]) for i in range(len(aspace))]  #making a list of mod angle bins to select over
    mod_angle_list.pop(0) 


    while run_num<run_num_tot//cpu_num:
        print('in while loop')
        run_num+=1
        #print()
        mod_angle_array_12=[]
        mod_angle_array_3=[]
        #print('{} core {} run'.format(iterable,run_num))
        #for mod_fac in modulation_factor_12:
#                   #print(mod_fac)
#
         #   cdf_12=(lambda phi: phi - modmin + 0.5*mod_fac*p * ( np.sin( 2*phi - 2*psi ) - np.sin( 2*modmin - 2*psi ) ))
         #   cdf_min_12=cdf_12(np.radians(modmin))
         #   cdf_max_12=cdf_12(np.radians(modmax))
         #   random_12=np.random.uniform(cdf_min_12,cdf_max_12)
         #   mod_angle_array_12.append(inversefunc(cdf_12, y_values=random_12))
            #print(mod_angle_array_12)

#              print('mod angles calculated')




       # for mod_fac in modulation_factor_3:
       #     cdf_3=(lambda i: (1/np.pi) * (i +  (mu*p*0.5) * (np.sin((2*i)-(2*psi)) + np.sin(2*psi))))
       #     cdf_min_3=cdf(np.radians(0))
       #     cdf_max_3=cdf(np.radians(180))
       #     random_3=np.random.uniform(cdf_min,cdf_max)
       #     mod_angle_array_3.append(inversefunc(cdf_3, y_values=random_3))


        av_power_G_array=[]
        m_array=[]
        av_power_array_mod_ps_12=[]
        av_power_array_mod_ps_3=[]
        av_power_array_real=[]
        av_power_array_im=[]
        #mod_angle_array_12=np.array(])

        print(len(mod_angles))
        print(len(TIME))
        for i in mod_angle_list:
            mod_min=i[0]
            print(np.degrees(mod_min))
            mod_max=i[1]

            print(np.degrees(mod_max))

            index_mod_angle=[mod_min<=k<=mod_max for k in mod_angles] #define the index over mod angle
            TIME_12=TIME[index_mod_angle] #selecting/indexing the photons that meet the criteria of this mod angle range

           # print('len data bin 12',len(data_bin_12))
            #TIME_12=data_bin_12['TIME']
            #print(TIME_12)

            #making subject lightcurve
            lc_12=Lightcurve.make_lightcurve(TIME_12,dt=bin_length,gti=GTI)
            lc_12.apply_gtis()

            #print('lc sub')

            #plt.figure()
            #plt.plot(lc_12.time,lc_12.counts)
            #plt.show()
            #making powerspectrum for errorbars
                #power spectrum used for frac rms errorbars
            ps_12=Powerspectrum.from_lightcurve(lc_12,seg_length,norm='frac')
            ps_3=Powerspectrum.from_lightcurve(lc_ref,seg_length,norm='frac')

            #making averagd cross spectrum
            cs = AveragedCrossspectrum.from_lightcurve(lc_12,lc_ref,seg_length,norm='frac') #making averagd cross spec
            mod_G=np.sqrt((cs.power.real)**2+(cs.power.imag**2))
            m=cs.m



            fourier_f=cs.freq
            real_power=cs.power.real   #real and im power of G
            im_power=cs.power.imag

           #averaging the modulus of G to use in drms

            #d_G_ = {'G': np.array(mod_G), 'fourier_freq': np.array(cs.freq)}
            #df_G = pd.DataFrame(data=d_G_)
            #selected_rows_G = df_G[(df_G['fourier_freq'] >= fmin) & (df_G['fourier_freq'] <= fmax)]

            #print(selected_rows_G[:1000])

            freq_index=list(locate(np.array(cs.freq), lambda x: fmin <= x <= fmax))

            mod_G_selected=mod_G[freq_index]
            #print(mod_G_selected[:1000])


            av_power_G=mod_G_selected.mean()
            #print('modulus of G mean: {}'.format(av_power_G))
            av_power_G_array.append(av_power_G)
            #np.savetxt('Results/av_G_mod'+str(source_name)+'_'+str(mod_bin_number)+'_bins_freqs_'+str(fmin)+'_'+str(fmax)+'_'+str(bin_length)+'_'+str(seg_length)+'.txt',av_power_G_array)




           #averaging the power spectrum over fourier freq for drms errorbars

            mod_ps_12=ps_12.power.real #this is because the poewr spectrum is real by definition!
            mod_ps_3=ps_3.power.real

                        #d_mod_ps_12 = {'mod_power_ps_12': np.array(mod_ps_12), 'fourier_freq': np.array(ps_12.freq)}
            #df_mod_ps_12 = pd.DataFrame(data=d_mod_ps_12)
            #selected_rows_mod_ps_12 = df_mod_ps_12[(df_mod_ps_12['fourier_freq'] >= fmin) & (df_mod_ps_12['fourier_freq'] <= fmax)]
            selected_rows_mod_ps_12=mod_ps_12[freq_index]
            av_power_mod_ps_12=selected_rows_mod_ps_12.mean()
            av_power_array_mod_ps_12.append(av_power_mod_ps_12)


            #d_mod_ps_3 = {'mod_power_ps_3': np.array(mod_ps_3), 'fourier_freq': np.array(ps_3.freq)}
            #df_mod_ps_3 = pd.DataFrame(data=d_mod_ps_3)
            #selected_rows_mod_ps_3 = df_mod_ps_3[(df_mod_ps_3['fourier_freq'] >= fmin) & (df_mod_ps_3['fourier_freq'] <= fmax)]
            selected_rows_mod_ps_3=mod_ps_3[freq_index]
            av_power_mod_ps_3=selected_rows_mod_ps_3.mean()
            av_power_array_mod_ps_3.append(av_power_mod_ps_3)



            #Average real power over frequency range
            #d_real = {'real_power': np.array(real_power), 'fourier_freq': np.array(cs.freq),'dPco':np.array(dPco)}
            #df_real = pd.DataFrame(data=d_real)
            #selected_rows_real = df_real[(df_real['fourier_freq'] >= fmin) & (df_real['fourier_freq'] <= fmax)]
            selected_rows_real=real_power[freq_index]
            av_power_real=selected_rows_real.mean()
            av_power_array_real.append(av_power_real)
            #np.savetxt('Results/G_av_real_'+str(source_name)+'_'+str(mod_bin_number)+'_bins_freqs_'+str(fmin)+'_'+str(fmax)+'_'+str(bin_length)+'_'+str(seg_length)+'.txt',av_power_array_real)


            #Average imaginary power over frequency
            #d_im = {'im_power': np.array(im_power), 'fourier_freq': np.array(cs.freq),'dPcoim': np.array(dPcoim)}
            #df_im = pd.DataFrame(data=d_im)
            #selected_rows_im = df_im[(df_im['fourier_freq'] >= fmin) & (df_im['fourier_freq'] <= fmax)]
            selected_rows_im=im_power[freq_index]
            av_power_im=selected_rows_im.mean()
            av_power_array_im.append(av_power_im)
            #np.savetxt('Results/G_av_im_'+str(source_name)+'_'+str(mod_bin_number)+'_bins_freqs_'+str(fmin)+'_'+str(fmax)+'_'+str(bin_length)+'_'+str(seg_length)+'.txt',av_power_array_im)


            #everything i want from a single run    
            #mod_G_av=np.sqrt(np.array(av_power_array_real)**2+np.array(av_power_array_im)**2)


            #complex_G=[av_power_array_real+1j*av_power_array_im for i in range(len(av_power_array_real))]
            complex_G=[complex(a,b) for a, b in zip(av_power_array_real, av_power_array_im)] 

            #print('{} core {} run complex G {}'.format(iterable,run_num,complex_G))
            #arg_G_av=np.vectorize(np.angle(complex_G))
            d_mod_G=np.sqrt((1/(2*(len(selected_rows_real)*cs.m)))*((av_power_mod_ps_12*av_power_mod_ps_3)-((av_power_mod_ps_3/(av_power_norm_real))*(av_power_G**2))))
            #d_arg_G_av=d_mod_G/av_power_G
            #print(d_mod_G)

        dG_running_sum+=d_mod_G**2 #squared so it can be stacked in quadrature
        G_running_sum+=complex_G
        #print('{} core {} run  Gsum {}'.format(iterable,run_num,G_running_sum))
        norm_power_running_sum+=av_power_norm_real
        #print('G sum',G_running_sum)
        #print('dG sum',dG_running_sum)
        #print('norm power sum',norm_power_running_sum)
        G_running_average=G_running_sum/(run_num)
        dG_running_average=np.sqrt(dG_running_sum)/(run_num)
        norm_power_running_average=norm_power_running_sum/(run_num)
        #print('average',G_running_average)
        #print('dG average',dG_running_average)
        #print('norm av',norm_power_running_average)


    np.savetxt(f'/mnt/zfsusers/ixpemel/new_null/{source_name}_{iterable}_stacked_av_G.txt',G_running_average)
    np.savetxt(f'/mnt/zfsusers/ixpemel/new_null/{source_name}_{iterable}_stacked_av_dG.txt',dG_running_average)
    np.savetxt(f'/mnt/zfsusers/ixpemel/new_null/{source_name}_{iterable}_stacked_av_norm_power.txt',norm_power_running_average)




            #complex_G=[av_power_array_real+1j*av_power_array_im for i in range(len(av_power_array_real))]
            complex_G=[complex(a,b) for a, b in zip(av_power_array_real, av_power_array_im)] 

            #print('{} core {} run complex G {}'.format(iterable,run_num,complex_G))
            #arg_G_av=np.vectorize(np.angle(complex_G))
            d_mod_G=np.sqrt((1/(2*(len(selected_rows_real)*cs.m)))*((av_power_mod_ps_12*av_power_mod_ps_3)-((av_power_mod_ps_3/(av_power_norm_real))*(av_power_G**2))))
            #d_arg_G_av=d_mod_G/av_power_G
            #print(d_mod_G)

        dG_running_sum+=d_mod_G**2 #squared so it can be stacked in quadrature
        G_running_sum+=complex_G
        #print('{} core {} run  Gsum {}'.format(iterable,run_num,G_running_sum))
        norm_power_running_sum+=av_power_norm_real
        #print('G sum',G_running_sum)
        #print('dG sum',dG_running_sum)
        #print('norm power sum',norm_power_running_sum)
        G_running_average=G_running_sum/(run_num)
        dG_running_average=np.sqrt(dG_running_sum)/(run_num)
        norm_power_running_average=norm_power_running_sum/(run_num)
        #print('average',G_running_average)
        #print('dG average',dG_running_average)
        #print('norm av',norm_power_running_average)


    np.savetxt(f'/mnt/zfsusers/ixpemel/new_null/{source_name}_{iterable}_stacked_av_G.txt',G_running_average)
    np.savetxt(f'/mnt/zfsusers/ixpemel/new_null/{source_name}_{iterable}_stacked_av_dG.txt',dG_running_average)
    np.savetxt(f'/mnt/zfsusers/ixpemel/new_null/{source_name}_{iterable}_stacked_av_norm_power.txt',norm_power_running_average)


